In [1]:
import numpy as np
import pandas as pd
import re
import spacy
import jsonlines
from spacy import displacy
from spacy.symbols import nsubj, VERB
from spacy.pipeline import EntityRuler
from spacy.matcher import Matcher
from spacy.tokens import Span



In [2]:
df2 = pd.read_excel("LinkedIn pessoas.xlsx")
df2['len'] = df2['descrição'].str.len()
df2 = df2[df2['len']>100]
cvs = df2['descrição']

In [37]:
df1 = pd.read_excel("LinkedIn jobs complete.xlsx")

df1['len'] = df1['description'].str.len()
df1 = df1[df1['len']>500]

jobs = df1['description']
df1.drop(['Unnamed: 0'],axis=1,inplace=True)
df1['job'] = df1['job'].str.split('|',expand=True)[0]
cargos = df1['job']
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623 entries, 0 to 742
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   job          598 non-null    object
 1   description  623 non-null    object
 2   len          623 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 19.5+ KB


In [27]:
class job_skills_parser:

    def __init__(self,nlp):
        nlp=nlp
        config = {"phrase_matcher_attr": None, "validate": True,
                  "overwrite_ents": False, "ent_id_sep": "||", }

        ruler = nlp.add_pipe("entity_ruler", config=config)
        ruler.from_disk('padrões')
        self.rules = ruler.patterns
        
    def update_patterns(self,new_patterns):
        ruler.add_patterns(patterns)
        self.rules = ruler.patterns
        ruler.to_disk('padrões')

    def fit_text(self, text):
        text = text.lower()
        text = text.replace('.', '. ')
        text = text.replace(',', ', ')
        text = text.replace(';', '; ')
        text = text.replace(':', ': ')
        text = text.replace(' . ', ' ')
        text = text.replace('  ', ' ')
        self.doc = nlp(text)

    def show_skills(self):
        displacy.render(self.doc, style="ent", jupyter=True)

    def get_skills(self):
        skills = [(ent.text, ent.label_)
                  for ent in self.doc.ents
                  if ent.label_ in ['JOB_DESC', 'hard_skills', 'formação', 'extra_skills', 'sistemas', 'experiencia']]

        return skills

    def compare_skills(self, text2):

        patterns_words = [
            'fazer','apuração','apurar','atuação','atuar','realização','realizar','elaoração','elaborar',
            'planejamento','planejar','apoio','apoiar','facilidade','revisão','revisar','execução','executar',
            'análise','analisar','validação','validar','participação','participar','confecção','confeccionar',
            'consolidação','consolidar','desenvolvimento','desenvolver','criação','criar','auxílio','avançado','intermediário'
            'auxiliar','acompanhamento','acompanhar','controle','controlar','revisão','revisar','básico','fluente',
            'formação','formação','formar','graduação','graduar','superior','cursar','cursando','graduação','graduar',
            'vivencia','vivência','atuar','atuação','experiencia','experiência','conhecimento','conhecer','habilidade',
            'capacidade','entendimento','entender','facilidade','avançado','intermediário','intermediario','basico']
        
        stopw = ['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'é', 'com', 'não', 'uma', 'os', 'no', 'se',
                 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'à', 'seu', 'sua',
                 'ou', 'ser', 'quando', 'muito', 'há', 'nos', 'já', 'está', 'eu', 'também', 'só', 'pelo', 'pela', 'até',
                 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me',
                 'esse', 'eles', 'estão', 'você', 'tinha', 'foram', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha',
                 'têm', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'será', 'nós', 'tenho', 'lhe', 'deles', 'essas',
                 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu',
                 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 
                 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive',
                 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja',
                 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos', 'estiverem', 
                 'hei', 'há', 'havemos', 'hão', 'houve', 'houvemos', 'houveram', 'houvera', 'houvéramos', 'haja', 
                 'hajamos', 'hajam', 'houvesse', 'houvéssemos', 'houvessem', 'houver', 'houvermos', 'houverem', 'houverei',
                 'houverá', 'houveremos', 'houverão', 'houveria', 'houveríamos', 'houveriam', 'sou', 'somos', 'são', 'era',
                 'éramos', 'eram', 'fui', 'foi', 'fomos', 'foram', 'fora', 'fôramos', 'seja', 'sejamos', 'sejam', 'fosse', 
                 'fôssemos', 'fossem', 'for', 'formos', 'forem', 'serei', 'será', 'seremos', 'serão', 'seria', 'seríamos', 
                 'seriam', 'tenho', 'tem', 'temos', 'tém', 'tinha', 'tínhamos', 'tinham', 'tive', 'teve', 'tivemos',
                 'tiveram', 'tivera', 'tivéramos', 'tenha', 'tenhamos', 'tenham', 'tivesse', 'tivéssemos', 'tivessem', 
                 'tiver', 'tivermos', 'tiverem', 'terei', 'terá', 'teremos', 'terão', 'teria', 'teríamos', 'teriam']
        
        
        palavras_comuns = [*patterns_words,*stopw]
        disable_palavras_comuns = []
        
        doc2 = nlp(text2)

        try:
            doc1_skills = self.get_skills()
        except:
            raise ValueError('não tem texto1 para fazer a comparação, provavelmente vc esqueceu da parte "fit_text"')

        doc2_skills = [(ent.text, ent.label_) for ent in doc2.ents if ent.label_ in
                       ['JOB_DESC', 'hard_skills', 'formação', 'extra_skills', 'sistemas', 'experiencia']]

        skills_matrix = []

        for sk_doc1, type_doc1 in doc1_skills:

            sim_ = []
            text_doc1 = ' '.join([w for w in sk_doc1.split() if w not in palavras_comuns])

            for sk_doc2, type_doc2 in doc2_skills:
                if type_doc2 == type_doc1:
                    text_doc2 = ' '.join([w.lower() for w in sk_doc2.split() if w not in palavras_comuns])
                    sim = nlp(text_doc1).similarity(nlp(text_doc2))
                    sim_.append(sim)    
                else: sim_.append(0)

            try:
                maxsim = np.array(sim_).argmax()
                if sim_[maxsim] > .8:
                    skills_matrix.append([type_doc1, sk_doc1, doc2_skills[maxsim][0], sim_[maxsim]])
                else:
                    skills_matrix.append([type_doc1, sk_doc1, 0, 0])
                
            except: 
                skills_matrix.append([type_doc1, sk_doc1, 0, 0])

        try:
            match = np.array([m[3] for m in skills_matrix])
        except:
            total_score = 0
            match = np.array(0)

        return skills_matrix, match.mean()



# Usando o modelo

In [28]:
#carregando o modelo com a inguagem PT large
nlp = spacy.load("pt_core_news_lg")
modelo = job_skills_parser(nlp)

Encontrando skills nos textos.
pipe: 

1) fit_text('texto da vaga ou do currículo',type=string)
   
2) usar as funções para de extração de features (skills)
   a) show_skills: cria um visual destacando os skills encontrados
   b) get_skills: cria uma lista de tuples com as skills encontradas
            
3) vc também pode comparar um texto com outro para ver as similaridades entre os skills encontrados, por exemplo,      comparar uma vaga de emprego com um currículo para ver o quão aderente são. para isso utilize a função comparecompare_skills

In [29]:
#encontrando skills em uma das vagas de emprego da Catho
modelo.fit_text(jobs[75])
print(len(jobs[75]))
modelo.show_skills()
modelo.get_skills()

1043


[('acompanhamento do fechamento contábil', 'JOB_DESC'),
 ('consolidação e análises de performance mensal e reporting de resultados e desvios orçado versus realizado',
  'JOB_DESC'),
 ('validações e consolidação de relatórios de resultado', 'JOB_DESC'),
 ('controle interno', 'JOB_DESC'),
 ('elaboração de dashboards com os kpis de desempenho', 'JOB_DESC'),
 ('elaboração de apresentações dos resultados para reuniões mensais com gerência',
  'JOB_DESC'),
 ('superior completo em ciências contábeis', 'formação'),
 ('superior completo em ciências contábeis', 'formação'),
 ('excel avançado', 'hard_skills'),
 ('power point', 'sistemas')]

In [30]:
#encontrando skills em um dos perfis do LinkedIn
modelo.fit_text(cvs[11])
modelo.show_skills()
modelo.get_skills()

[('formação em ciencias contábeis', 'formação'),
 ('inglês avançado', 'hard_skills'),
 ('experiência em contabilidade', 'experiencia'),
 ('acompanhando diretamente e em interface com outras áreas', 'JOB_DESC'),
 ('experiência com controladoria e processos de contabilidade societária/regulatória',
  'experiencia'),
 ('controle de imobilizado e controle de processos internos', 'JOB_DESC'),
 ('elaboração de notas explicativas', 'JOB_DESC'),
 ('elaboração dos reportes das centrais eólicas', 'JOB_DESC'),
 ('elaboração das demonstrações financeiras', 'JOB_DESC'),
 ('consolidação dos resultados', 'JOB_DESC'),
 ('controle do imobilizado de acordo com as normas regulatórias e societárias',
  'JOB_DESC'),
 ("controle de odi's e odd's", 'JOB_DESC'),
 ('elaboração das obrigações regulatórias enviadas à aneel', 'JOB_DESC'),
 ('controles internos e memorandos', 'JOB_DESC'),
 ('consolidadas do grupo', 'JOB_DESC'),
 ('consolidação das empresas do grupo', 'JOB_DESC'),
 ('elaboração do transfer', 'JOB_D

In [31]:
#comparando uma vaga fake com um perfil coletado
#modelo.fit_text('experiência em controladoria, experiencia em contabilidade, formação em ciências contábeis, conhecimento em SAP,inglês intermediário')
modelo.fit_text(jobs[68])
result = modelo.compare_skills(cvs[1])
print(result[1])
pd.DataFrame(result[0])

0.43232235691508525


,0,1,2,3
0,JOB_DESC,desenvolvimento pessoal contrata para cargo de...,0,0.000000
1,experiencia,atuar em tempo integral,0,0.000000
2,JOB_DESC,elaboração de processo de forecast mensal,elaboração dos relatórios de auditoria,0.815285
3,JOB_DESC,revisões trimestrais,0,0.000000
4,JOB_DESC,controle de despesas por,controle de despesas de viagens,0.867161
5,JOB_DESC,análise das variações entre real e orçado de r...,0,0.000000
6,JOB_DESC,elaboração do orçamento conforme definição de ...,elaboração dos relatórios de auditoria,0.827988
7,JOB_DESC,elaboração de relatórios gerenciais,elaboração dos relatórios de auditoria,0.948145


In [44]:
modelo.fit_text(jobs[67])
for cv in cvs:
    result = modelo.compare_skills(cv)
    print(result[1])


0.0
0.9429451086509233
0.0
0.0
0.9555847055425972


<ipython-input-37-bcb1a77f46bf>:93: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim = nlp(text_doc1).similarity(nlp(text_doc2))


0.0
0.4123573895496698
0.0
0.9460070964047893
0.0
0.0
0.8678714530666218


In [48]:
#comparando várias vagas com todos os perfis


njob = 0

scores = []

for job in jobs:
    njob += 1
    npessoa = 0
    modelo.fit_text(job)
    for pessoa in cvs:
        npessoa += 1
        resultado = modelo.compare_skills(pessoa)
        scores.append([njob,npessoa,resultado[1]])

    

<ipython-input-27-bcb1a77f46bf>:93: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim = nlp(text_doc1).similarity(nlp(text_doc2))
<ipython-input-27-bcb1a77f46bf>:113: RuntimeWarning: Mean of empty slice.
  return skills_matrix, match.mean()
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [49]:
df = pd.DataFrame(scores)
df = df.rename(columns={0:'Vaga',1:'Perfil',2:'score'})
pivot_score = df.pivot_table('score','Vaga','Perfil')
pivot_score

Perfil,1,2,3,4,5,6,7,8,9,10,11,12,13
Vaga,,,,,,,,,,,,,
1,0.044874,0.144309,0.055556,0.0,0.000000,0.148419,0.000000,0.0,0.099020,0.000000,0.000000,0.148510,0.0
2,0.000000,0.151081,0.000000,0.0,0.148371,0.000000,0.000000,0.0,0.227358,0.000000,0.000000,0.000000,0.0
3,0.047432,0.147504,0.000000,0.0,0.048346,0.048636,0.000000,0.0,0.144294,0.158695,0.000000,0.097631,0.0
4,0.305937,0.328796,0.000000,0.0,0.205836,0.102464,0.106632,0.0,0.337556,0.000000,0.000000,0.211401,0.0
5,0.117039,0.243392,0.000000,0.0,0.251052,0.120839,0.119276,0.0,0.243307,0.000000,0.235884,0.250464,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,0.000000,0.200615,0.000000,0.0,0.000000,0.201361,0.000000,0.0,0.000000,0.000000,0.432662,0.000000,0.0
620,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.174046,0.000000,0.0
621,0.000000,0.078903,0.000000,0.0,0.040022,0.039809,0.000000,0.0,0.040022,0.000000,0.043130,0.040038,0.0


In [50]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(10,200))
sns.heatmap(pivot_score,annot=True,cbar=False,cmap="magma")

<AxesSubplot:xlabel='Perfil', ylabel='Vaga'>

In [39]:
cargos[0]

'Analista de Controladoria Sênior - Foco Modelagem de Seguros/Resseguros'

In [45]:
all_skills = {'job':[],'skills':[],'tipo':[]}
n=0
for text in jobs:
    modelo.fit_text(text)
    for skill in modelo.get_skills():
        all_skills['job'].append(cargos[n])
        all_skills['skills'].append(skill[0])
        all_skills['tipo'].append(skill[1])
    n=+1

In [46]:
df = pd.DataFrame(all_skills)

In [47]:
df.to_excel('Skills_linkedin.xlsx')


# Calibrar os padrões

In [52]:
patterns = [{'label': 'JOB_DESC',
  'pattern': [{'LEMMA': {'IN': [
      'fazer',
      'apuração','apurar',
      'atuação','atuar',
      'realização','realizar',
      'elaboração','elaborar',
      'planejamento','planejar',
      'apoio','apoiar',
      'facilidade',
      'revisão','revisar',
      'execução','executar',
      'análise','analisar',
      'validação','validar',
      'participação','participar',
      'confecção','confeccionar',
      'consolidação','consolidar',
      'desenvolvimento','desenvolver',
      'criação','criar',
      'auxílio','auxiliar',
      'acompanhamento','acompanhar',
      'controle','controlar','controlo',
      'revisão','revisar'
      ]}},
   {'POS': {'IN': ['VERB', 'ADP', 'DET','NOUN', 'ADJ','ADV', 'PROPN','CCONJ',]},'OP': '+'}
             ]},
            
            
 {'label': 'formação',
  'pattern': [{'LEMMA': {'IN': ['formação',
      'formação','formar',
      'graduação','graduar',
      'superior',
      'cursar','cursando',
      'graduação','graduar'
      ]}},
   {'LOWER': {'IN': [
       'superior',
      'completo',''
      'cursando',
      'graduação',
      ]},
    'OP': '*'},
   {'POS': 'ADP'},
   {'POS': {'IN': ['VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'PROPN', 'CCONJ']},
    'OP': '+'}]},
            
            
 {'label': 'experiencia',
  'pattern': [{'LEMMA': {'IN': [
      'vivencia','vivência',
      'atuar','atuação',
      'experiencia','experiência'
        ]}},              
   {'POS': {'IN': ['ADP', 'DET']}},
   {'POS': {'IN': ['VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'PROPN', 'NUM','CCONJ']},
    'OP': '+'}]},
            
 {'label': 'hard_skills',
  'pattern': [{'LEMMA': {'IN': [
      'conhecimento','conhecer',
      'habilidade',
      'capacidade',
      'entendimento','entender',
      'facilidade'
        ]}},
   {'POS': {'IN': ['ADP', 'DET']}},
   {'POS': {'IN': ['VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'PROPN', 'NUM']},
    'OP': '+'}]},
            
            
 {'label': 'hard_skills',
  'pattern': [{'IS_PUNCT': False},
   {'LOWER': {'IN': [
      'avançado',
      'intermediário',
      'intermediario',
      'basico',
      'básico',
      'fluente'
   ]}},
   {'POS': {'IN': ['VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'PROPN']},
    'OP': '*'}]},
            
            
 {'label': 'sistemas',
  'pattern': [{'LOWER': {'IN': ['power',
      'PBI',
      'powerbi',
      'sap',
      'totvs',
      'tableau',
      'pacote',
      'hyperion',
      'office',
            ]}},  
   {'LOWER': {'IN': ['bi',
      'powerbi',
      'sap',
      'totvs',
      'tableau',
      'office',
      'point',
      'query']},
    'OP': '*'}]},
            
 {'label': 'sistemas',
  'pattern': [{'LOWER': 'sistema'},
   {'POS': {'IN': ['VERB', 'ADP', 'NOUN', 'ADJ', 'PROPN']}, 'OP': '+'}]}

            
 {'label': 'sistemas',
  'pattern': [{'LOWER': {'IN': [

    'DAX'
    'python',
    'vba',
    'macro',
    'sql',
    'JavaScript',
    'Java',
    'c#',
    'c++',
    'PHP',
    'Ruby',
    'CSS',
    'TypeScript',
    'Swift',
    'Objective-C',
    'Scala',
    'R',
    'Go',
    'Shell',
    'PowerShell',
    'Perl',
    'Kotlin',
    'Haskell',
      
  ]}},  
   ]}]





SyntaxError: invalid syntax (<ipython-input-52-f85791cca2d0>, line 112)

In [ ]:
nlp = spacy.load("pt_core_news_lg")
nlp=nlp
config = {"phrase_matcher_attr": None, "validate": True,
                  "overwrite_ents": False, "ent_id_sep": "||", }

ruler = nlp.add_pipe("entity_ruler", config=config)
ruler.add_patterns(patterns)
ruler.to_disk('padrões')

Preprocessing for the model